Bin CMIP6 3D meteorological variable data by distance from sea ice edges in a selected Arctic region

In [9]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.interpolate import griddata
import regionmask

In [11]:
# Load model cloud fraction data and ensure time is CF-compliant
f = xr.open_dataset('/home/rcostell/Model_Files/CMIP6/Historical/Cloud_Fraction_3D/Raw_Output/clcalipso_CFmon_GFDL-CM4_historical_r1i1p1f1_gr1_195001-201412.nc')
f['time'] = xr.decode_cf(f)['time']

f = f.sel(time=slice('2007', '2014'))
f = f.resample(time='1M').mean()

lat0 = 60
lat1 = 90

# Convert longitudes to [-180, 180] for consistent mapping
f.coords['lon'] = (f.coords['lon'] + 180) % 360 - 180
sorted_indices = np.argsort(f.lon.values)
f = f.isel(lon=sorted_indices)

f = f.sel(lat = slice(lat0, lat1))
lon_plot = f.lon.values
lat_plot = f.lat.values

# Apply a land mask to retain ocean grid cells only
land_mask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(lon_plot, lat_plot)
f = f.where(land_mask != 0)

# Load filtered and regridded sea ice concentration data - can be a raw file as well
filename_si_nh = '/home/rcostell/Model_Files/CMIP6/Historical/Filtered_Sea_Ice_Concentration/GFDL_Filtered_Sea_Ice.nc'
f_si_nh = xr.open_dataset(filename_si_nh)
f_si_nh['time'] = xr.decode_cf(f_si_nh)['time']

# Compute monthly means
f_si_nh = f_si_nh.sel(time=slice('2007', '2014'))
f_si_nh = f_si_nh.resample(time='1M').mean()

# Standardize longitudes to [-180, 180] and ensure increasing order
f_si_nh.coords['lon'] = (f_si_nh.coords['lon'] + 180) % 360 - 180
f_si_nh = f_si_nh.sortby(f_si_nh.lon)

# Interpolate sea ice data to the same grid as the cloud fraction dataset
f_si_nh = f_si_nh.interp(lat=lat_plot, lon=lon_plot)

si_winter_nh = f_si_nh.sel(lat=slice(lat0, lat1))

land_mask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(lon_plot, lat_plot)

f0 = si_winter_nh.where(land_mask != 0)

# Match sea ice time steps to the cloud fraction dataset
ind = []
for i, d in enumerate(f.time.values):
    tmp = np.where(f0.time.values == d)[0]
    ind.append(tmp[0])

ind = np.array(ind)

f0 = f0.isel(time = ind).sea_ice_concentration
f = f.assign(seaice_conc=f0)

#Attaching another 3D variable like air temperature
f1 = xr.open_dataset('/home/rcostell/Model_Files/CMIP6/Historical/Air_Temperature/Raw_Output/ta_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_195001-201412.nc')

f1['time'] = xr.decode_cf(f1)['time']
f1 = f1.sel(time=slice('2007', '2014'))
f1 = f1.resample(time='1M').mean()

f1.coords['lon'] = (f1.coords['lon'] + 180) % 360 - 180
f1 = f1.interp(lat=lat_plot, lon=lon_plot)
f1 = f1.sel(lat=slice(lat0, lat1))

xgrid_nh = f1.lon.values
ygrid_nh = f1.lat.values
land_mask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(xgrid_nh, ygrid_nh)
f1 = f1.where(land_mask != 0)

f1 = f1.isel(time=ind).ta
f1['time'] = f.time
f = f.assign(air=f1)

f = f.sel(lon=slice(-15, 55))

ice_achv = f.seaice_conc.values/100

#Extract key variables
cl_achv = f.clcalipso.values
air_achv = f.air.values

#Retrieve coordinate and dimension arrays
lat = f.lat.values
lon = f.lon.values
time = f.time.values 
level = f.plev.values
altitude = f.alt40.values/1000.

# Define array dimensions for later loops or reshaping
nlat = len(lat)
nlon = len(lon)
ntime = len(time)

<string>:7: FutureWarning: 'M' is deprecated and will be removed in a future version. Please use 'ME' instead of 'M'.
<string>:7: FutureWarning: 'M' is deprecated and will be removed in a future version. Please use 'ME' instead of 'M'.
<string>:7: FutureWarning: 'M' is deprecated and will be removed in a future version. Please use 'ME' instead of 'M'.


In [13]:
def Binning_distance_from_ice_edge(ice_achv, var_achv, ntime, nlat, nlon, hem, 
                                   outputcount=False, model=True):
    """
    Bins a 2D or 3D meteorological variable (e.g., cloud fraction) relative to 
    the sea ice edge for each time step and longitude.

    Parameters
    ----------
    ice_achv : np.ndarray
        Sea ice concentration array with dimensions [time, lat, lon].
        Values should range from 0 to 1 (0% to 100% divided by 100).
    var_achv : np.ndarray
        Meteorological variable (e.g., cloud fraction, temperature) array. 
        Expected dimensions are:
          - [time, lat, lon] for 2D variables, or 
          - [time, level, lat, lon] for 3D variables.
    ntime : int
        Number of time steps to process.
    nlat : int
        Number of latitude points.
    nlon : int
        Number of longitude points.
    hem : str
        Hemisphere flag: 
          - 'nh' for Northern Hemisphere (sea ice = 1 → ice-covered)
          - 'sh' for Southern Hemisphere (sea ice = 1 → ocean-covered)
        Used to determine which side of the ice edge is "ice" vs "ocean".
    outputcount : bool, optional
        If True, also return the count array that tracks valid grid points 
        contributing to each latitude bin (default: False).
    model : bool, optional
        Set to True for model data (default). If False, allows flexibility 
        for observational data input (currently handled the same way).

    Returns
    -------
    var_out : np.ndarray
        Mean value of the binned meteorological variable along the ice-edge-normal direction. 
        Shape depends on input:
          - (2 * nlat, ntime) for 2D variable input
          - (vertical_levels, 2 * nlat, ntime) for 3D variable input
    ice_out : np.ndarray
        Corresponding mean sea ice fraction in each bin.
    count_out : np.ndarray, optional
        (Returned only if outputcount=True)
        Number of valid samples contributing to each bin.
    """

    for j in range(ntime):  # Loop over time steps
        # Select the sea ice field for this time
        if model:
            ice = ice_achv[j, :, :]
        else:
            ice = ice_achv[j, :, :]  # (Same here, placeholder for observational logic)

        # Extract variable field for the same time step
        var = var_achv[j]  # e.g., cloud fraction or temperature
        
        ndim = var.ndim  # Check if var is 2D (lat, lon) or 3D (level, lat, lon)

        # Loop over longitudes to analyze latitudinal structure
        for i in range(nlon):
            tmp = ice[:, i]  # Sea ice concentration along a latitude column

            if ndim == 3:
                # For 3D variables, select vertical profiles at this longitude
                var_tmp = var[:, :, i]
                nh = var_tmp.shape[0]  # Number of vertical levels
            else:
                # For 2D variables, select latitudinal transect at this longitude
                var_tmp = var[:, i]

            # Identify indices of sea ice vs. open ocean based on a 50% threshold
            if hem == 'nh':
                ind1 = np.where(tmp <= 0.5)[0]  # Ice-covered points
                ind2 = np.where(tmp > 0.5)[0]   # Open-water points
            else:
                ind1 = np.where(tmp > 0.5)[0]
                ind2 = np.where(tmp <= 0.5)[0]

            n1 = ind1.size  # Number of ice points
            n2 = ind2.size  # Number of open-water points

            if i == 0:
                tmp0 = np.full((2 * nlat), -999.)
                tmp0[nlat - n1 : nlat] = tmp[ind1]   # Ice-covered section
                tmp0[nlat : nlat + n2] = tmp[ind2]   # Open-water section

                count0 = np.full((2 * nlat), 1.)     # Initialize counts

            if ndim == 3:
                # For 3D fields: allocate (level × latitude) array
                var_tmp0 = np.full((nh, 2 * nlat), -999.)
                var_tmp0[:, nlat - n1 : nlat] = var_tmp[:, ind1]
                var_tmp0[:, nlat : nlat + n2] = var_tmp[:, ind2]
            else:
                # For 2D fields (lat × lon)
                var_tmp0 = np.full((2 * nlat), -999.)
                var_tmp0[nlat - n1 : nlat] = var_tmp[ind1]
                var_tmp0[nlat : nlat + n2] = var_tmp[ind2]

        # Stack additional longitudes along the third dimension
        else:
            tmp1 = np.full((2 * nlat), -999.)
            tmp1[nlat - n1 : nlat] = tmp[ind1]
            tmp1[nlat : nlat + n2] = tmp[ind2]

            count1 = np.full((2 * nlat), 1.)

            if ndim == 3:
                var_tmp1 = np.full((nh, 2 * nlat), -999.)
                var_tmp1[:, nlat - n1 : nlat] = var_tmp[:, ind1]
                var_tmp1[:, nlat : nlat + n2] = var_tmp[:, ind2]
            else:
                var_tmp1 = np.full((2 * nlat), -999.)
                var_tmp1[nlat - n1 : nlat] = var_tmp[ind1]
                var_tmp1[nlat : nlat + n2] = var_tmp[ind2]

            # Combine new longitude slice with accumulated data
            tmp0 = np.dstack((tmp0, tmp1))
            count0 = np.dstack((count0, count1))
            var_tmp0 = np.dstack((var_tmp0, var_tmp1))

        # Replace placeholder fill values with NaNs for later averaging
        var_tmp0[var_tmp0 == -999.] = np.nan
        tmp0[tmp0 == -999.] = np.nan

        if ndim == 2:
            count0[np.isnan(var_tmp0)] = 0.
        
        # Average over longitudes to collapse the stacked dimension
        count00 = nlon * np.nanmean(count0[0], axis=1)
        tmp00 = np.nanmean(tmp0[0], axis=1)

        # For 3D variables (with vertical levels), average over longitude
        # Otherwise, average over latitude for 2D data
        if ndim == 3:
            var_tmp00 = np.nanmean(var_tmp0, axis=2)
        else:
            var_tmp00 = np.nanmean(var_tmp0[0], axis=1)

        # Initialize output arrays on the first time step
        if j == 0:
            count_out = count00
            var_out = var_tmp00
            ice_out = tmp00
        else:
            # Stack new time-step results along a new axis
            count_out = np.dstack((count_out, count00))        
            var_out = np.dstack((var_out, var_tmp00))
            ice_out = np.dstack((ice_out, tmp00))

    # Remove dummy dimension added during stacking
    count_out = count_out[0]
    ice_out = ice_out[0]

    # For 2D variables, remove redundant first index
    if ndim != 3:
        var_out = var_out[0]
    
    # Return depending on variable dimensionality and output settings
    if ndim == 3:
        return var_out, ice_out
    else:
        if outputcount:
            return var_out, ice_out, count_out    
        else:
            return var_out, ice_out

In [14]:
print("Start extracting")
print(cl_achv.shape)

cl_out, ice_out = Binning_distance_from_ice_edge(ice_achv, cl_achv, ntime, nlat, nlon, 'nh')
air_out, ice_out = Binning_distance_from_ice_edge(ice_achv, air_achv, ntime, nlat, nlon, 'nh')

print("Binning done")

# --- Convert binned outputs to xarray DataArrays ---
# Define the coordinate system: “distance” from ice edge and “time”.
# The distance coordinate runs from negative (sea-ice side) to positive (open-ocean side) values.
ds_tmp = xr.DataArray(cl_out, coords=[altitude, np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["altitude", "distance","time"]).rename('clcalipso')

ds_tmp1 = xr.DataArray(ice_out, coords=[np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["distance","time"]).rename('seaice_conc')

ds_tmp2 = xr.DataArray(air_out, coords=[level,np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["level","distance","time"]).rename('air')

# --- Combine all DataArrays into a single dataset ---
ds_out = xr.merge([ds_tmp, ds_tmp1, ds_tmp2])

#ds_out.to_netcdf("Binned_3D_Meteorology.nc")
print('Job accomplished')

Start extracting
(96, 40, 30, 56)
Binning done
Job accomplished


/tmp/ipykernel_571307/1835411665.py:133: RuntimeWarning: Mean of empty slice
  tmp00 = np.nanmean(tmp0[0], axis=1)
/tmp/ipykernel_571307/1835411665.py:138: RuntimeWarning: Mean of empty slice
  var_tmp00 = np.nanmean(var_tmp0, axis=2)
